#### Queries - Exploración de Tablas DuckLake
##### Configuración
> Asegúrate de tener un archivo `.env` en esta carpeta con las credenciales necesarias.


In [1]:
# Instalar dependencias si es necesario
# !pip install duckdb python-dotenv

In [2]:
import os
import duckdb
from dotenv import load_dotenv

# Cargar variables de entorno desde .env
load_dotenv(dotenv_path='.env', override=True)

# Configuración desde variables de entorno
POSTGRES_HOST = os.getenv('POSTGRES_HOST')
POSTGRES_PORT = os.getenv('POSTGRES_PORT', '5432')
POSTGRES_DB = os.getenv('POSTGRES_DB')
POSTGRES_USER = os.getenv('POSTGRES_USER')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')

S3_ENDPOINT = os.getenv('S3_ENDPOINT', 'rustfs:9000')
RUSTFS_USER = os.getenv('RUSTFS_USER', 'admin')
RUSTFS_PASSWORD = os.getenv('RUSTFS_PASSWORD')
RUSTFS_SSL = os.getenv('RUSTFS_SSL', 'false')
RUSTFS_BUCKET = os.getenv('RUSTFS_BUCKET', 'mitma')

print("✅ Variables de entorno cargadas")


✅ Variables de entorno cargadas


In [3]:
# Crear conexión a DuckDB
con = duckdb.connect()

# Instalar y cargar extensiones necesarias
critical_extensions = ['ducklake', 'postgres', 'httpfs']
for ext in critical_extensions:
    try:
        con.execute(f"INSTALL {ext};")
        con.execute(f"LOAD {ext};")
        print(f"✅ Extensión {ext} cargada")
    except Exception as e:
        print(f"⚠️ Advertencia al cargar {ext}: {e}")
        try:
            con.execute(f"LOAD {ext};")
            print(f"✅ Extensión {ext} cargada (ya estaba instalada)")
        except Exception as e2:
            print(f"❌ Error al cargar {ext}: {e2}")

# Cargar extensión spatial (opcional)
try:
    con.execute("INSTALL spatial;")
    con.execute("LOAD spatial;")
    print("✅ Extensión spatial cargada")
except Exception as e:
    print(f"⚠️ Spatial extension no disponible: {e}")


✅ Extensión ducklake cargada
✅ Extensión postgres cargada
✅ Extensión httpfs cargada
✅ Extensión spatial cargada


In [4]:
# Configurar S3 para RustFS
con.execute(f"SET s3_endpoint='{S3_ENDPOINT}';")
con.execute(f"SET s3_access_key_id='{RUSTFS_USER}';")
con.execute(f"SET s3_secret_access_key='{RUSTFS_PASSWORD}';")
con.execute(f"SET s3_use_ssl={RUSTFS_SSL};")
con.execute("SET s3_url_style='path';")

print("✅ Configuración de S3/RustFS completada")


✅ Configuración de S3/RustFS completada


In [ ]:
# ⚠️ PELIGRO: Borrar todo el contenido de mitmav2/main/silver_od en S3
# Ejecutar solo cuando necesites limpiar datos corruptos antes de reingestar

import boto3
from botocore.client import Config

# Configurar cliente S3 con las credenciales de RustFS
endpoint_parts = S3_ENDPOINT.split(':')
s3_host = endpoint_parts[0]
s3_port = endpoint_parts[1] if len(endpoint_parts) > 1 else '9000'
s3_endpoint_url = f"http://{s3_host}:{s3_port}" if RUSTFS_SSL.lower() == 'false' else f"https://{s3_host}:{s3_port}"

s3_client = boto3.client(
    's3',
    endpoint_url=s3_endpoint_url,
    aws_access_key_id=RUSTFS_USER,
    aws_secret_access_key=RUSTFS_PASSWORD,
    config=Config(signature_version='s3v4')
)

# Bucket y prefijo a borrar
bucket_name = 'mitmav2'
prefix = 'main/silver_od/'  # Cambia a 'main/silver_od_v2/' si es tu tabla de pruebas

print(f"🗑️  Borrando contenido de s3://{bucket_name}/{prefix}...")

try:
    paginator = s3_client.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name, Prefix=prefix)
    
    deleted_count = 0
    for page in pages:
        if 'Contents' in page:
            objects_to_delete = [{'Key': obj['Key']} for obj in page['Contents']]
            if objects_to_delete:
                s3_client.delete_objects(Bucket=bucket_name, Delete={'Objects': objects_to_delete})
                deleted_count += len(objects_to_delete)
                print(f"   ✅ Borrados {len(objects_to_delete)} objetos...")
    
    print(f"✅ Total borrados: {deleted_count} objetos de s3://{bucket_name}/{prefix}" if deleted_count > 0 else f"ℹ️ No se encontraron objetos")
except Exception as e:
    print(f"❌ Error: {e}")


In [5]:
# Attach DuckLake con PostgreSQL Catalog
postgres_connection_string = f"""
    dbname={POSTGRES_DB} host={POSTGRES_HOST} user={POSTGRES_USER} password={POSTGRES_PASSWORD} port={POSTGRES_PORT} 
    sslmode=require connect_timeout=30 keepalives=1 keepalives_idle=30 keepalives_interval=10 keepalives_count=5 tcp_user_timeout=30000
"""

# Verificar si ducklake ya está attachado
try:
    databases = con.execute("SELECT database_name FROM duckdb_databases();").fetchdf()
    if 'ducklake' not in databases['database_name'].values:
        attach_query = f"""
            ATTACH 'ducklake:postgres:{postgres_connection_string}' AS ducklake (DATA_PATH 's3://{RUSTFS_BUCKET}/');
        """
        con.execute(attach_query)
        print("✅ DuckLake attachado correctamente")
    else:
        print("✅ DuckLake ya está attachado")
except Exception as e:
    print(f"⚠️ Advertencia al attachar DuckLake: {e}")

# Usar la base de datos ducklake
con.execute("USE ducklake;")
print("✅ Usando base de datos ducklake")

✅ DuckLake attachado correctamente
✅ Usando base de datos ducklake


In [15]:
# 🔍 DIAGNÓSTICO: Ver qué produce strptime y las funciones de partición
# Esto nos dirá exactamente qué valores está generando

SQL("""
    SELECT 
        fecha as fecha_raw,
        periodo as periodo_raw,
        typeof(fecha) as tipo_fecha,
        -- Conversión paso a paso
        strptime(fecha, '%Y%m%d') as fecha_parsed,
        strptime(fecha, '%Y%m%d')::TIMESTAMP as fecha_timestamp,
        strptime(fecha, '%Y%m%d')::TIMESTAMP + (periodo::INTEGER * INTERVAL 1 HOUR) as fecha_final,
        -- Valores que irían a las particiones
        year(strptime(fecha, '%Y%m%d')::TIMESTAMP + (periodo::INTEGER * INTERVAL 1 HOUR)) as partition_year,
        month(strptime(fecha, '%Y%m%d')::TIMESTAMP + (periodo::INTEGER * INTERVAL 1 HOUR)) as partition_month,
        day(strptime(fecha, '%Y%m%d')::TIMESTAMP + (periodo::INTEGER * INTERVAL 1 HOUR)) as partition_day
    FROM bronze_mitma_od_municipios
    LIMIT 10
""")


,fecha_raw,periodo_raw,tipo_fecha,fecha_parsed,fecha_timestamp,fecha_final,partition_year,partition_month,partition_day
0,20230425,13,VARCHAR,2023-04-25,2023-04-25,2023-04-25 13:00:00,2023,4,25
1,20230425,13,VARCHAR,2023-04-25,2023-04-25,2023-04-25 13:00:00,2023,4,25
2,20230425,13,VARCHAR,2023-04-25,2023-04-25,2023-04-25 13:00:00,2023,4,25
3,20230425,13,VARCHAR,2023-04-25,2023-04-25,2023-04-25 13:00:00,2023,4,25
4,20230425,13,VARCHAR,2023-04-25,2023-04-25,2023-04-25 13:00:00,2023,4,25
5,20230425,13,VARCHAR,2023-04-25,2023-04-25,2023-04-25 13:00:00,2023,4,25
6,20230425,13,VARCHAR,2023-04-25,2023-04-25,2023-04-25 13:00:00,2023,4,25
7,20230425,13,VARCHAR,2023-04-25,2023-04-25,2023-04-25 13:00:00,2023,4,25
8,20230425,13,VARCHAR,2023-04-25,2023-04-25,2023-04-25 13:00:00,2023,4,25
9,20230425,13,VARCHAR,2023-04-25,2023-04-25,2023-04-25 13:00:00,2023,4,25


In [16]:
# 🔍 DIAGNÓSTICO: Ver qué datos tiene silver_od_v2 y sus valores de fecha
# Esto nos dirá si los datos insertados tienen fechas correctas o no

SQL("""
    SELECT 
        fecha,
        typeof(fecha) as tipo,
        year(fecha) as y,
        month(fecha) as m,
        day(fecha) as d,
        COUNT(*) as cnt
    FROM silver_od_v2
    GROUP BY fecha, typeof(fecha), year(fecha), month(fecha), day(fecha)
    ORDER BY fecha
    LIMIT 20
""")


RuntimeError: Query interrupted

In [6]:
# Wrapper SQL para simplificar consultas y liberar memoria
import gc

class SQL:
    """Wrapper para ejecutar queries SQL de forma simple y liberar memoria"""
    def __init__(self, connection):
        self.con = connection
    
    def __call__(self, query):
        """Ejecuta una query y retorna el resultado como DataFrame (se muestra automáticamente en notebooks).
        Libera memoria de DuckDB después de ejecutar la query."""
        result = self.con.execute(query).fetchdf()
        # Liberar memoria de DuckDB después de fetchdf
        try:
            # Forzar liberación de memoria de DuckDB
            self.con.execute("PRAGMA force_checkpoint")
        except:
            pass

        gc.collect()
        return result
    
    def fetch_df(self, query):
        """Ejecuta una query y retorna un DataFrame"""
        result = self.con.execute(query).fetchdf()
        try:
            self.con.execute("PRAGMA force_checkpoint")
        except:
            pass
        return result

# Crear instancia global SQL
SQL = SQL(con)
print("✅ Wrapper SQL creado. Usa SQL(query) para ejecutar consultas - el DataFrame se mostrará automáticamente")
print("✅ El wrapper libera memoria automáticamente después de cada consulta")


✅ Wrapper SQL creado. Usa SQL(query) para ejecutar consultas - el DataFrame se mostrará automáticamente
✅ El wrapper libera memoria automáticamente después de cada consulta


In [13]:
SQL("SELECT * FROM bronze_mitma_od_municipios USING SAMPLE 10")

,fecha,periodo,origen,destino,distancia,actividad_origen,actividad_destino,estudio_origen_posible,estudio_destino_posible,residencia,renta,edad,sexo,viajes,viajes_km,loaded_at,source_file
0,20230425,13,08121,08015,10-50,trabajo_estudio,no_frecuente,no,no,08,10-15,NA,NA,4.585,84.469,2026-01-01 21:13:50.400860+01:00,https://movilidad-opendata.mitma.es/estudios_b...
1,20230425,09,08015,08040,10-50,casa,no_frecuente,no,no,08,<10,45-65,mujer,2.269,79.02,2026-01-01 21:13:50.400860+01:00,https://movilidad-opendata.mitma.es/estudios_b...
2,20230425,14,11033,11004,10-50,trabajo_estudio,casa,no,no,11,<10,25-45,mujer,20.221,272.324,2026-01-01 21:13:50.400860+01:00,https://movilidad-opendata.mitma.es/estudios_b...
3,20230425,13,08101,08073,0.5-2,casa,frecuente,no,no,08,10-15,45-65,mujer,42.771,45.478,2026-01-01 21:13:50.400860+01:00,https://movilidad-opendata.mitma.es/estudios_b...
4,20230425,13,08085_AM,08305,2-10,frecuente,frecuente,no,no,08,>15,45-65,hombre,3.976,20.458,2026-01-01 21:13:50.400860+01:00,https://movilidad-opendata.mitma.es/estudios_b...
5,20230425,13,08051,08051,2-10,casa,frecuente,no,no,08,10-15,65-100,NA,16.078,39.666,2026-01-01 21:13:50.400860+01:00,https://movilidad-opendata.mitma.es/estudios_b...
6,20230425,13,08156,08096,2-10,trabajo_estudio,no_frecuente,no,no,08,10-15,45-65,mujer,1.967,16.034,2026-01-01 21:13:50.400860+01:00,https://movilidad-opendata.mitma.es/estudios_b...
7,20230425,14,14024_AM,14038,10-50,frecuente,casa,no,no,14,<10,25-45,mujer,8.458,149.382,2026-01-01 21:13:50.400860+01:00,https://movilidad-opendata.mitma.es/estudios_b...
8,20230425,07,48003,48040_AM,10-50,casa,trabajo_estudio,no,no,48,10-15,45-65,mujer,4.856,101.529,2026-01-01 21:13:50.400860+01:00,https://movilidad-opendata.mitma.es/estudios_b...
9,20230425,07,50181_AM,50017_AM,10-50,casa,trabajo_estudio,no,no,50,10-15,NA,NA,2.655,78.034,2026-01-01 21:13:50.400860+01:00,https://movilidad-opendata.mitma.es/estudios_b...


In [21]:
print("📊 Listando TODAS las tablas en la base de datos:")
SQL("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'main'
    ORDER BY table_name;
""")

📊 Listando TODAS las tablas en la base de datos:


,table_name
0,_temp_od_enriched
1,bronze_ine_empresas_municipio
2,bronze_ine_municipios
3,bronze_ine_poblacion_municipio
4,bronze_ine_renta_municipio
5,bronze_mitma_ine_relations
6,bronze_mitma_municipios
7,bronze_mitma_od_municipios
8,bronze_mitma_overnight_stay_municipios
9,bronze_mitma_people_day_municipios


In [22]:
# Filtrar tablas que comienzan con 'silver_' usando SQL wrapper
SQL("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'main'
        AND table_name LIKE 'silver_%'
    ORDER BY table_name;
""")

,table_name
0,silver_ine_all
1,silver_mitma_distances
2,silver_mitma_ine_mapping
3,silver_od
4,silver_od_all
5,silver_od_processed_dates
6,silver_od_quality
7,silver_overnight_stay
8,silver_people_day
9,silver_zones


In [23]:
# silver_zones
SQL("SELECT * FROM silver_zones USING SAMPLE 10")

,id,nombre,geometry_obj,centroid
0,06020_AM,Bienvenida agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 242, 41, 198, 192, 22...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,20079,Zarautz,"[3, 4, 0, 0, 0, 0, 0, 0, 70, 213, 10, 192, 69,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,03024,Benejúzar,"[3, 4, 0, 0, 0, 0, 0, 0, 205, 6, 85, 191, 253,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,13033_AM,Castellar de Santiago agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 51, 254, 73, 192, 139...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,48095_AM,Zaldibar agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 89, 168, 34, 192, 74,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
5,14041,Montemayor,"[3, 4, 0, 0, 0, 0, 0, 0, 98, 4, 149, 192, 72, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
6,28164,Valdetorres de Jarama,"[3, 4, 0, 0, 0, 0, 0, 0, 4, 205, 96, 192, 199,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
7,46220,Sagunto/Sagunt,"[3, 4, 0, 0, 0, 0, 0, 0, 38, 202, 142, 190, 32...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
8,11039,Vejer de la Frontera,"[3, 4, 0, 0, 0, 0, 0, 0, 212, 33, 190, 192, 21...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
9,36035,Nigrán,"[3, 4, 0, 0, 0, 0, 0, 0, 88, 163, 12, 193, 248...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [24]:
# silver_od
SQL("SELECT * FROM silver_od USING SAMPLE 10")

,fecha,origen_zone_id,destino_zone_id,viajes,viajes_km,residencia
0,2023-03-06 02:00:00,18182,18035_AM,5.754,57.128,29
1,2023-03-06 18:00:00,07036,07036,2683.466,7056.702,07
2,2023-03-06 13:00:00,25016_AM,25189,60.485,256.584,25
3,2023-03-06 18:00:00,28010,28061,147.168,859.073,28
4,2023-03-06 06:00:00,24089,24175_AM,192.054,1212.216,24
5,2023-03-06 22:00:00,08159,08120,7.540,147.940,08
6,2023-03-06 20:00:00,46078,46250,1892.113,7167.581,46
7,2023-03-08 08:00:00,27022,15076,16.000,746.596,27
8,2023-03-06 18:00:00,18165,18021,17.230,138.193,18
9,2023-03-06 16:00:00,08108,08096,241.013,1006.632,08


In [25]:
# silver_od_all
SQL("SELECT * FROM silver_od_all USING SAMPLE 10")

,fecha,origen_zone_id,destino_zone_id,viajes,viajes_km,distancia
0,2022-03-02 14:00:00,07046,07026,9.280,135.612,10-50
1,2022-03-02 20:00:00,08135,08296_AM,4.239,19.791,2-10
2,2022-03-02 02:00:00,15002,15002,3.713,1.927,0.5-2
3,2022-03-02 13:00:00,25072,25240_AM,3.867,167.835,10-50
4,2022-03-02 06:00:00,08019,08263,2.326,21.258,2-10
5,2022-03-02 07:00:00,38011,38006,4.474,212.490,10-50
6,2022-03-02 15:00:00,15031,15058,3.069,27.526,2-10
7,2022-03-02 11:00:00,12031,12084,3.337,33.029,2-10
8,2022-03-02 13:00:00,08180,08124,1.952,12.565,2-10
9,2022-03-02 09:00:00,04075,04017_AM,7.538,224.822,10-50


In [37]:
SQL("""
    SELECT COUNT(*) FROM silver_od
    UNION ALL
    SELECT COUNT(*) FROM bronze_mitma_od_municipios
    UNION ALL
    SELECT COUNT(*) FROM silver_od_quality
""")

,count_star()
0,321102467
1,321547012
2,378844701


In [ ]:
# Buscar viajes con formato x-y (número-número) en la columna distancia
# El patrón busca valores como "0.5-2", "2-10", "10-50", etc.
SQL("""
    SELECT *
    FROM bronze_mitma_od_municipios
    WHERE distancia REGEXP '[0-9]+\\.?[0-9]*-[0-9]+\\.?[0-9]*'
    ORDER BY fecha
    LIMIT 100
""")

<>:6: SyntaxWarning: invalid escape sequence '\.'
<>:6: SyntaxWarning: invalid escape sequence '\.'
/var/folders/wf/tb7y7pjd0619fvty47rsc_fw0000gn/T/ipykernel_61184/4081752675.py:6: SyntaxWarning: invalid escape sequence '\.'
  WHERE viajes REGEXP '[0-9]+\.?[0-9]*-[0-9]+\.?[0-9]*'
/var/folders/wf/tb7y7pjd0619fvty47rsc_fw0000gn/T/ipykernel_61184/4081752675.py:6: SyntaxWarning: invalid escape sequence '\.'
  WHERE viajes REGEXP '[0-9]+\.?[0-9]*-[0-9]+\.?[0-9]*'


ParserException: Parser Error: syntax error at or near "REGEXP"

LINE 4:     WHERE viajes REGEXP '[0-9]+\.?[0-9]*-[0-9]+\.?[0-9]*'
                         ^

In [26]:
# silver_od_quality
SQL("SELECT * FROM silver_od_quality USING SAMPLE 10")


,fecha,origen_zone_id,destino_zone_id,residencia,viajes_per_capita,km_per_capita,flag_negative_viajes,flag_negative_viajes_km,z_viajes_per_capita,flag_outlier_viajes_per_capita
0,2023-03-06 10:00:00,46032,46164,46,0.001378,0.010284,False,False,-0.609927,False
1,2023-03-06 07:00:00,48054,48902,48,0.019364,0.046328,False,False,-0.165481,False
2,2023-03-06 13:00:00,08187,08019,08,0.005505,0.090030,False,False,-0.507960,False
3,2023-03-06 09:00:00,08136,08096,08,0.014461,0.078092,False,False,-0.286633,False
4,2023-03-06 13:00:00,17049_AM,17023,17,0.000604,0.024027,False,False,-0.629061,False
5,2023-03-06 20:00:00,50298,50297,50,0.018196,0.479649,False,False,-0.194352,False
6,2023-03-07 20:00:00,08282,08110,08,0.000668,0.029805,False,False,-0.627481,False
7,2023-03-07 10:00:00,30030,30038,30,0.001276,0.011240,False,False,-0.612464,False
8,2023-03-07 20:00:00,11004,11004,11,0.139137,0.310054,False,False,2.794223,False
9,2023-03-07 14:00:00,08203,08235,08,0.025386,0.085218,False,False,-0.016681,False


In [27]:
# silver_overnight_stay
SQL("SELECT * FROM silver_overnight_stay USING SAMPLE 10")


,fecha,zona_pernoctacion,zona_residencia,personas
0,2023-03-02,28079,2702804_AD,53.827
1,2023-03-02,28145,28009,3.039
2,2023-03-02,41038,28014,14.980
3,2023-03-01,27005,27024_AM,7.306
4,2023-03-02,34056_AM,0105902,3.170
5,2023-03-02,16078,0200303,7.500
6,2023-03-02,49178_AM,4718610,12.549
7,2023-03-02,39082_AM,4802006,4.137
8,2023-03-01,27021_AM,1503006,9.185
9,2023-03-01,02003,23082_AM,22.500


In [28]:
# silver_people_day
SQL("SELECT * FROM silver_people_day USING SAMPLE 10")


,fecha,zona_pernoctacion,edad,sexo,numero_viajes,personas
0,2023-03-02,06159,25-45,hombre,2+,155.203
1,2023-03-02,08136,65-100,mujer,1,48.698
2,2023-03-02,08282,0-25,hombre,1,27.397
3,2023-03-02,11015,65-100,mujer,1,184.052
4,2023-03-02,12031,65-100,mujer,1,12.456
5,2023-03-02,15009,65-100,hombre,2,302.083
6,2023-03-01,03064,65-100,hombre,0,264.871
7,2023-03-01,28168_AM,0-25,hombre,2,64.415
8,2023-03-01,22907_AM,65-100,hombre,2,130.622
9,2023-03-01,25204_AM,45-65,mujer,2,171.379


In [29]:
# silver_mitma_distances
SQL("SELECT * FROM silver_mitma_distances USING SAMPLE 10")


,origin,destination,distance_km
0,02045,02073_AM,15.915524
1,02045,04029,293.576838
2,03043,05002_AM,475.104811
3,04064,05168_AM,473.471941
4,01051,14049,657.831267
5,03053,17164_AM,505.905016
6,12902_AM,17169_AM,378.949676
7,08097_AM,20022_AM,511.946709
8,08088,23050,797.580336
9,08125,23055,749.165117


In [30]:
# silver_mitma_ine_mapping
SQL("SELECT * FROM silver_mitma_ine_mapping USING SAMPLE 10")


,nombre,codigo_ine,municipio_mitma
0,barbadillo de herreros,09037,09330_AM
1,bohonal de ibor,10030,10140_AM
2,fuentes de ayodar,12064,12072_AM
3,algar de mesa,19016,19086_AM
4,zufre,21079,21069_AM
5,garganta de los montes,28062,28027_AM
6,barillas,31048,31068_AM
7,malpartida,37178,37174_AM
8,"riba de escalote, la",42157,42035_AM
9,canizal,49034,49081_AM


In [31]:
# silver_ine_all
SQL("SELECT * FROM silver_ine_all USING SAMPLE 10")


,id,nombre,empresas,renta_media,poblacion_total,poblacion_hombres,poblacion_mujeres,year
0,08068,Cervelló,803.0,18664.0,9743.0,4911.0,4832.0,
1,10116_AM,Malpartida de Plasencia agregacion de municipios,319.0,11008.0,5236.0,2608.0,2628.0,
2,13092_AM,Villanueva de la Fuente agregacion de municipios,402.0,11341.0,6142.0,3198.0,2944.0,
3,19143,Horche,126.0,13502.0,3119.0,1596.0,1523.0,
4,22907_AM,Aínsa-Sobrarbe agregacion de municipios,424.0,17618.0,5222.0,2712.0,2510.0,
5,28137,"Santos de la Humosa, Los",126.0,13849.0,2867.0,1486.0,1381.0,
6,37156,Guijuelo,525.0,13971.0,5427.0,2776.0,2651.0,
7,40906_AM,San Cristóbal de Segovia agregacion de municipios,388.0,14781.0,5796.0,2955.0,2841.0,
8,50099,Épila,219.0,12570.0,4704.0,2431.0,2273.0,
9,50219,"Puebla de Alfindén, La",708.0,14581.0,6550.0,3356.0,3194.0,


In [32]:
# silver_zones_all
SQL("SELECT * FROM silver_zones_all USING SAMPLE 10")


,id,nombre,geometry_obj,centroid
0,07001,Alaró,"[3, 4, 0, 0, 0, 0, 0, 0, 125, 90, 51, 64, 230,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,07033,Manacor,"[3, 4, 0, 0, 0, 0, 0, 0, 119, 17, 79, 64, 34, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,20036,Hondarribia,"[3, 4, 0, 0, 0, 0, 0, 0, 129, 180, 233, 191, 4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,29090_AM,Tolox agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 55, 113, 157, 192, 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,08149_AM,Olost agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 194, 162, 6, 64, 131,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
5,36038,Pontevedra,"[3, 4, 0, 0, 0, 0, 0, 0, 235, 234, 9, 193, 170...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
6,45118_AM,Novés agregacion de municipios,"[3, 4, 0, 0, 0, 0, 0, 0, 199, 76, 136, 192, 16...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
7,15024,Cerceda,"[3, 4, 0, 0, 0, 0, 0, 0, 16, 189, 7, 193, 42, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
8,47165,Tordesillas,"[3, 4, 0, 0, 0, 0, 0, 0, 216, 47, 160, 192, 24...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
9,12027,Benicarló,"[3, 4, 0, 0, 0, 0, 0, 0, 170, 67, 206, 62, 111...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
